In [23]:
!pip install qwikidata
!pip install pyarango

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import qwikidata
from qwikidata.json_dump import WikidataJsonDump

path = 'D:\wikidata.json'
wjd = WikidataJsonDump(path)

In [5]:
import pyArango.connection as CON
from pyArango.graph import *
db = CON.Connection(username="root",password="").databases["_system"]

In [6]:
item = []
prop = []
prop_lst = []
invalid_item = 0
invalid_prop = 0
for ii, entity_dict in enumerate(wjd):
    if ii>4000:
        break
    if entity_dict['type']=='property':
        try:
            a = entity_dict['labels']['en']['value']
            b = entity_dict['descriptions']['en']['value']
            prop_lst.append(entity_dict['id'])
        except:
            c = 1
    
for ii, entity_dict in enumerate(wjd):
    dct = {}
    if ii>4000:
        break
    if entity_dict['type']=='item':
        try:
            dct['_key'] = entity_dict['id']
            dct['type'] = entity_dict['type']
            dct['label'] = entity_dict['labels']['en']['value']
            dct['description'] = entity_dict['descriptions']['en']['value']
            aa = entity_dict['claims'].keys()
            temp = []
            for j in aa:
                if j in prop_lst:
                    temp.append(j)
            dct['claims'] = temp
            item.append(dct)
        except:
            invalid_item = invalid_item + 1
    else:
        try:
            dct['_key'] = entity_dict['id']
            dct['type'] = entity_dict['type']
            dct['label'] = entity_dict['labels']['en']['value']
            dct['description'] = entity_dict['descriptions']['en']['value']
            prop.append(dct)
            
        except:
            invalid_prop = invalid_prop + 1

In [8]:
t2_item = db.collections["WikiItems"]
t2_prop = db.collections["WikiProps"]
for i in item:
    doc = t2_item.createDocument()
    for j in i.keys():
        doc[j] = i[j]
    doc.save()
for i in prop:
    doc = t2_prop.createDocument()
    for j in i.keys():
        doc[j] = i[j]
    doc.save()
print(len(item))
print(len(prop))

3685
284


In [9]:
edge_v3 = db.collections["WikiEdge"]
for i in item:
    for j in i['claims']:
        if j in prop_lst:
            raw = edge_v3.createEdge()
            raw['_from'] = f"item/{i['_key']}"
            raw['_to'] = f"prop/{j}"
            raw['prop'] = t2_prop.fetchDocument(j,rawResults=True)['label']
#             print(raw)
            raw.save()

In [18]:
print(prop)

[{'_key': 'P16', 'type': 'property', 'label': 'highway system', 'description': 'system of which the highway is a part'}, {'_key': 'P19', 'type': 'property', 'label': 'place of birth', 'description': 'the most specific known (e.g. city instead of country)'}, {'_key': 'P22', 'type': 'property', 'label': 'father', 'description': 'male parent'}, {'_key': 'P31', 'type': 'property', 'label': 'instance of', 'description': 'this item is a concrete example of this class, category or object group'}, {'_key': 'P35', 'type': 'property', 'label': 'head of state', 'description': 'official with the highest formal authority in a country'}, {'_key': 'P36', 'type': 'property', 'label': 'capital', 'description': 'location (city, municipality) of governmental seat of the country, or administrative territorial entity'}, {'_key': 'P37', 'type': 'property', 'label': 'official language', 'description': 'language designated as official by this item'}, {'_key': 'P39', 'type': 'property', 'label': 'position held